<a href="https://colab.research.google.com/github/jumafernandez/clasificacion_correos/blob/main/notebooks/jcc/01-BERT-beto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo a evaluar JCC: BERT

En esta notebook se presentan los experimentos sobre la estrategia de representación y técnica de aprendizaje *avanzada* utilizada para las JCC de  la Universidad Nacional de La Plata.

Para ello vamos a tomar el texto de la consulta de los correos y aplicar BERT con el modelo pre-entrenado en español (BETO - Departamento de Computación de la Universidad de Chile).


# Instalación de la librería *simpletransformers*

In [1]:
%%capture

# install simpletransformers
!pip install simpletransformers

# Elección de un modelo monolenguaje pre-entrenado

La librería *simpletransformers* se basa en la librería *Transformers* de HuggingFace. Esto permite utilizar todos los modelos pre-entrenados disponibles en la [Transformers library](https://huggingface.co/transformers/pretrained_models.html) que son provistos por toda la comunidad de desarrolladores. Para ver cuales son los modelos disponibles, se puede ingresar a [https://huggingface.co/models](https://huggingface.co/models).

En nuestro caso, vamos a utilizar el modelo `dccuchile/bert-base-spanish-wwm-cased`. Este modelo está pre-entrenado por un equipo de investigadores del Departamento de Computación de la Universidad de Chile.

# Carga del dataset con los correos

In [2]:
# Cargamos el archivo con las consultas que está en Github
from os import path

# En caso que no esté el archivo en Colab lo traigo
if not(path.exists('03-Correos_variables_estaticas.csv')):
  !wget https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/data/03-Correos_variables_estaticas.csv

In [3]:
# Leemos el archivo en un dataframe
import pandas as pd

# Cargamos los datos
df = pd.read_csv('03-Correos_variables_estaticas.csv', delimiter="|")

# Seleccionamos solo la consulta y la clase
df = df[["Consulta", "Clase"]]
df.head()

,Consulta,Clase
0,"hola quiero anotarme a las materias ,para el s...",Vacunas Enfermería
1,hola buenos días! quería saber cuando voy a po...,Vacunas Enfermería
2,hola quisiera saber si en la consulta de situa...,Situación Académica
3,buenas noches. en mi situacion academica apare...,Situación Académica
4,"hola, quisiera obtener mi promedio o saber co...",Situación Académica


In [4]:
# Definición de la cantidad de clases (el resto se agrupa en OTRAS CONSULTAS)
CANTIDAD_CLASES = 5

In [5]:
# Transformamos todas las Clases minoritarias (Puedo ir variando la cantidad de clases que derivo a la Clase "Otras Consultas")
clases = df.Clase.value_counts()
clases_minoritarias = clases.iloc[CANTIDAD_CLASES-1:].keys().to_list()
df.Clase[df['Clase'].isin(clases_minoritarias)] = "Otras Consultas"

# Se numeriza la clase
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['Clase']=le.fit_transform(df['Clase'])

# Me guardo las etiquetas de las clases (numerizadas)
class_list=le.classes_

# Clases balanceadas
df.head()

,Consulta,Clase
0,"hola quiero anotarme a las materias ,para el s...",2
1,hola buenos días! quería saber cuando voy a po...,2
2,hola quisiera saber si en la consulta de situa...,2
3,buenas noches. en mi situacion academica apare...,2
4,"hola, quisiera obtener mi promedio o saber co...",2


In [12]:
# Clases balanceadas
df.Clase.value_counts()

2    330
0    240
1    232
4    129
3     69
Name: Clase, dtype: int64

# Separación en train/test del dataset

In [6]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.10)

print('train shape: ',train_df.shape)
print('test shape: ',test_df.shape)

train shape:  (900, 2)
test shape:  (100, 2)


# Carga de BETO, el modelo pre-entrenado por el *dccuchile*

In [7]:
from simpletransformers.classification import ClassificationModel

# Definimos los hiperparámetros
# VER: maxlen=192
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 4}

# Create a ClassificationModel
model = ClassificationModel(
    "bert", 'dccuchile/bert-base-spanish-wwm-cased',
    num_labels=CANTIDAD_CLASES,
    args=train_args
)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased a

# Entrenamiento del modelo con el dataset

In [8]:
# Train the model
model.train_model(train_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:446: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/113 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running Epoch 1 of 4:   0%|          | 0/113 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/113 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/113 [00:00<?, ?it/s]

(452, 0.43901470874537163)

In [9]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score)

result

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1026: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

{'acc': 0.8,
 'eval_loss': 0.8326853480404959,
 'f1': 0.8000000000000002,
 'mcc': 0.7391304482407073}

# Prueba del modelo con un ejemplo real

In [10]:
test_correo = "Necesito tramitar mi boleto universitario"

predictions, raw_outputs = model.predict([test_correo])

print(class_list[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

Boleto Universitario


# Persistencia y carga del modelo

Se guarda el modelo en */outputs* y luego se carga

In [11]:
EJECUCION_PERSISTENCIA = 0

if EJECUCION_PERSISTENCIA:
  import os
  import tarfile

  def save_model(model_path='',file_name=''):
    files = [files for root, dirs, files in os.walk(model_path)][0]
    with tarfile.open(file_name+ '.tar.gz', 'w:gz') as f:
      for file in files:
        f.add(f'{model_path}/{file}')

  # Se guarda
  save_model('outputs','prueba_clasificador_correos')

  # RELOADED
  # Se descomprime para cargarse
  !tar -zxvf ./prueba_clasificador_correos.tar.gz
  !rm -rf outputs

  import os
  import tarfile

  def unpack_model(model_name=''): 
    tar = tarfile.open(f"{model_name}.tar.gz", "r:gz")
    tar.extractall()
    tar.close()

  unpack_model('prueba_clasificador_correos')

  # Se carga el modelo propio
  from simpletransformers.classification import ClassificationModel

  # define hyperparameter
  train_args ={"reprocess_input_data": True,
              "overwrite_output_dir": True,
              "fp16":False,
              "num_train_epochs": 4}

  # Create a ClassificationModel
  model = ClassificationModel(
      "bert", "outputs/",
      num_labels=CANTIDAD_CLASES,
      args=train_args
  )
  from sklearn.metrics import f1_score, accuracy_score

  def f1_multiclass(labels, preds):
      return f1_score(labels, preds, average='micro')
      
  result, model_outputs, wrong_predictions = model.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score)

  result

# Referencias
- https://towardsdatascience.com/bert-text-classification-in-a-different-language-6af54930f9cb
- https://medium.com/dair-ai/beto-spanish-bert-420e4860d2c6